Este jupyter notebook nos ayudará a entender qué es hacer web scraping en una página web desde un punto de vista práctico. El contenido de este jupyter notebook ha sido adaptado del capítulo 8 del libro de texto ***Getting Started with Beautiful Soup*** de Vineeth G. Nair (2014).

# Primer Paso: 
## Definir cuál es la página web desde donde vamos a extraer los datos

Aquí vamos a utilizar la página web [packtpub.com](https://www.packtpub.com/). Sobre esta página el scraper que construiremos nos permitirá extraer los datos del precio de una lista de libros vendidos en esa página web y buscar en otras páginas como [amazon.com](www.amazon.com) o [barnes and nobles](www.barnesandnobles.com) empleando el ISBN de cada libro (el ISBN de un libro es el equivalente a la cédula de identidad de una persona. Ayuda a identificar inequívocamente al libro).

# Segundo Paso:

## Explorar el contenido de la página web 

En este caso, solo vamos a concentrarnos en el título de cada libro, su precio de venta y su ISBN. Para explorar el contenido, primero ingresamos a la página web con un navegador y miramos su contenido como lo hacemos usualmente.

In [1]:
import urllib.request
import re
from bs4 import BeautifulSoup
packtpub_url = 'www.packtpub.com/all-products/all-books/'

In [2]:
def get_bookurls(url):
    page = urllib.request.urlopen(url)
    soup_packtpage = BeautifulSoup(page, 'lxml')
    page.close()
    next_page_li = soup_packtpage.find("li", class_="pager-nextlast")
    if next_page_li is None:
        next_page_url = None
    else:
            next_page_url = packtpub_url+next_page_li.a.get('href')
            
    return next_page_url

In [3]:
start_url = "www.packtpub.com/all-products/all-books/"
continue_scrapping = True
books_url = [start_url]
while continue_scrapping:
    next_page_url = get_bookurls(start_url)
    if next_page_url is None:
        continue_scraping = False
    else:
        books_url.append(next_page_url)
    start_url = next_page_url

ValueError: unknown url type: 'www.packtpub.com/all-products/all-books/'

In [4]:
def get_bookdetails(url):
    page = urllib2.urlopen(url)
    soup_packtpage = BeautifulSoup(page,"lxml")
    page.close()
    all_books_table = soup_packtpage.find("table",class_="views-view-grid")
    all_book_titles =all_books_table.find_all("div",class_="views-field-title")
    isbn_list = []
    for book_title in all_book_titles:
        book_title_span = book_title.span
        print("Title Name:"+book_title_span.a.string)
        print("Url:"+book_title_span.a.get('href'))
        price = book_title.find_next("div",class_="views-field-sell-price")
        print("PacktPub Price:"+price.span.string)
        isbn_list.append(get_isbn(book_title_span.a.get('href')))
    return isbn_list

In [5]:
def get_isbn(url):
    book_title_url = packtpub_url + url
    page = urllib2.urlopen(book_title_url)
    soup_bookpage = BeautifulSoup(page,"lxml")
    page.close()
    isbn_tag = soup_bookpage.find('b',text=re.compile("ISBN :"))
    return isbn_tag.next_sibling